### This notebook contains code for using the gpu only model.
The conda environment required for running this notebook can be installed and activated by running the following on the command line from within this folder: 
<code>conda env create -f evolution_env_gpu_only.yml </code>  \
<code>conda activate seq</code>


In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np

import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
import tensorflow as tf
from sklearn.decomposition import PCA
import pickle
import time
import tensorflow as tf
import numpy as np
from sklearn.manifold import MDS
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import matplotlib
import multiprocessing as mp

In [2]:

model_conditions = 'pTpA_Glu' #  'SC_Ura''pTpA_Glu' 


def closest_point(node, nodes):
    nodes = np.asarray(nodes)
    deltas = nodes - node
    dist_2 = np.einsum('ij,ij->i', deltas, deltas)
    return np.argmin(dist_2)



%matplotlib inline
import matplotlib.cm as cm
import pickle , csv ,  argparse , os ,random , multiprocessing , copy , scipy , multiprocessing
import tensorflow as tf, sys, numpy as np, h5py, pandas as pd
from tqdm import tqdm
import seaborn as sns
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from sklearn import preprocessing
from deap import creator, base, tools, algorithms

from os.path import splitext,exists,dirname,join,basename


def seq2feature(data,mapper,worddim):
    
    transformed = np.zeros([data.shape[0],1,len(data[0]),4] , dtype=np.bool )
    for i in range(data.shape[0]) :
        for j,k in enumerate(data[i]):
            #print j,k
            transformed[i,0,j] = mapper[k] 
            #print mapper[k]
    return transformed
    


def get_rc(A):
    A_r = np.flip(A,2)
    A = A_r
    A_onehot = np.array([1,0,0,0] ,  dtype=np.bool)
    C_onehot = np.array([0,1,0,0] ,  dtype=np.bool)
    G_onehot = np.array([0,0,1,0] ,  dtype=np.bool)
    T_onehot = np.array([0,0,0,1] ,  dtype=np.bool)
    N_onehot = np.array([0,0,0,0] ,  dtype=np.bool)

    for i in range(A.shape[0]) : 
        for j in range(A.shape[2]) : 
            if np.array_equal(A[i][0][j] , A_onehot) :
                A[i][0][j] = T_onehot
            elif np.array_equal(A[i][0][j] , C_onehot) :
                A[i][0][j] = G_onehot
            elif np.array_equal(A[i][0][j] , G_onehot) :
                A[i][0][j] = C_onehot
            elif np.array_equal(A[i][0][j] , T_onehot) :
                A[i][0][j] = A_onehot
            elif np.array_equal(A[i][0][j] , N_onehot) :
                A[i][0][j] = N_onehot

                
    return A






def hamming_distance(s1, s2):
    return sum(ch1 != ch2 for ch1,ch2 in zip(s1,s2))


# this function modified from Effective Tensorflow: https://github.com/vahidk/EffectiveTensorflow#multi_gpu
def make_parallel(fn, num_gpus, **kwargs):
    in_splits = {}
    for k, v in kwargs.items():
        in_splits[k] = tf.split(v, num_gpus)

    out_split=[]
    for i in range(num_gpus):
        with tf.device(tf.DeviceSpec(device_type="GPU", device_index=i)):
            with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
                out_split.append(fn(**{k : v[i] for k, v in in_splits.items()}))

    out_splits=[]

    out_splits.append(tf.concat([x[0] for x in out_split], axis=0))
    out_splits.append(tf.stack([x[1] for x in out_split], axis=0))
    return out_splits







def getPredictions(sequences_raw , sess , graph, X, X_rc, Y, model_cost, model_output ):

    BASEORDER = list("ACGT")
    A_onehot = np.array([1,0,0,0] ,  dtype=np.bool)
    C_onehot = np.array([0,1,0,0] ,  dtype=np.bool)
    G_onehot = np.array([0,0,1,0] ,  dtype=np.bool)
    T_onehot = np.array([0,0,0,1] ,  dtype=np.bool)
    N_onehot = np.array([0,0,0,0] ,  dtype=np.bool)

    mapper = {'A':A_onehot,'C':C_onehot,'G':G_onehot,'T':T_onehot,'N':N_onehot}
    worddim = len(mapper['A'])
 
    


    if(np.ndim(sequences_raw)==1) : 
        if(len(sequences_raw[0])==80) : 
            batchX = [];
            for ind in range(len(sequences_raw)) :
                individual_pred = ''.join(['T','G','C','A','T','T','T','T','T','T','T','C','A','C','A','T','C'] + sequences_raw[ind] + ['G','G','T','T','A','C','G','G','C','T','G','T','T'] )
                batchX.append(individual_pred)

        else :
            batchX = sequences_raw

        _teX = seq2feature(np.asarray(batchX))#,mapper,worddim)
        _teX_rc = get_rc(list(_teX)) 
        _teY = np.zeros([np.shape(_teX)[0] , 1] , dtype=float)

        
    elif(np.ndim(sequences_raw)==2) : 
        if(len(sequences_raw[0])==80) : 
            batchX = [];
            for ind in range(len(sequences_raw)) :
                individual_pred = ''.join(['T','G','C','A','T','T','T','T','T','T','T','C','A','C','A','T','C'] + sequences_raw[ind] + ['G','G','T','T','A','C','G','G','C','T','G','T','T'] )
                batchX.append(individual_pred)

        else :
            batchX = sequences_raw

        _teX = seq2feature(np.asarray(batchX))#,mapper,worddim)
        _teX_rc = get_rc(list(_teX)) 
        _teY = np.zeros([np.shape(_teX)[0] , 1] , dtype=float)

        
    



    batch_size = (np.shape(_teX)[0])
    #print batch_size
    jump_size =4096
    (q,r) = divmod(batch_size , jump_size)
    #print q , r 
    i=0
    teX_output = []
    while(i <= q ) :
        if(i< q  ) :
            #with tf.device(tf.DeviceSpec(device_type="CPU", device_index= 3 )):
            temp_result_step1=sess.run([model_output], feed_dict={X: _teX[jump_size*i:jump_size*i+jump_size,:], X_rc: _teX_rc[jump_size*i:jump_size*i+jump_size,:]  ,Y: _teY[jump_size*i:jump_size*i+jump_size,:]}) 
            temp_result_step2=[float(x) for x in temp_result_step1[0]]
            #print temp_result_step2

            teX_output = teX_output + temp_result_step2
            i = i+1

        elif (i==q) :
            #with tf.device('/device:CPU:3'):
            temp_result_step1 = sess.run([model_output], feed_dict={X: _teX[jump_size*i:,:], X_rc: _teX_rc[jump_size*i:,:]  ,Y: _teY[jump_size*i:,:]})
            temp_result_step2=[float(x) for x in temp_result_step1[0]]
            #print "here"
            teX_output = teX_output + temp_result_step2
            i = i+1


            #tf.reset_default_graph()
    return teX_output



    
_best_model_file = join('..',model_conditions,'models','best_model.ckpt')

_best_model_file_hyper = join('..',model_conditions,'models','hyper_search','best_model.ckpt')


tf.reset_default_graph()
sess = tf.Session()
sess.list_devices()
init=tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init)

new_saver = tf.train.import_meta_graph('%s.meta'%_best_model_file)
new_saver.restore(sess, tf.train.latest_checkpoint(join('..',model_conditions,'models')))
graph = tf.get_default_graph()




X = graph.get_tensor_by_name('Placeholder:0')
X_rc = graph.get_tensor_by_name('Placeholder_1:0')
Y = graph.get_tensor_by_name('Placeholder_2:0')

model_cost = graph.get_tensor_by_name('out/model_cost:0')
model_output = graph.get_tensor_by_name('out/model_output:0')



#####



if (1) : 
    with open('../pTpA_Glu/justSeqs_native_HQ.sequencedRegion110.NATIVE_ONLY.txt') as f: ### Replace with path to the sequences whose expression you want to compute
        reader = csv.reader(f, delimiter="\t")
        d = list(reader)	

    sequences = [di[1] for di in d]
    for i in range(0,len(sequences)) : 
        if (len(sequences[i]) > 110) :
            sequences[i] = sequences[i][-110:]
        if (len(sequences[i]) < 110) : 
            #print i
            while (len(sequences[i]) < 110) :
                sequences[i] = 'N'+sequences[i]


    seqdata = list(np.asarray(sequences))
    native_sequences=seqdata

if(0) : ### Only useful if you have processed data from the preprocessing script in this folder
    ## Load the data matrix 
    with h5py.File(join('..',model_conditions,'_teY.h5'), 'r') as hf:
        _teY = hf['_teY'][:]
    #expressions.shape

    ## Load the sequences 
    with h5py.File(join('..',model_conditions,'_teX.h5'), 'r') as hf:
        _teX = hf['_teX'][:]
    #onehot_sequences.shape

    ## Load the rc sequences 
    with h5py.File(join('..',model_conditions,'_teX_rc.h5'), 'r') as hf:
        _teX_rc = hf['_teX_rc'][:]

    seqdata = _teX









###### Used to be a function
n=1024
full_seqdata_length= np.shape(seqdata)[0]
sequence_length = len(seqdata[0]) - 30 # 80 , 17 on 5' end and 13 on 3' end 

seqs = np.asarray(seqdata)     # you need to convert to np array to index the sequences properly

np.random.seed(1234)
seqs = seqs[np.random.choice(full_seqdata_length,n)]

sequences = seqs 
expressions = np.asarray(getPredictions(original_sequences , sess , graph, X, X_rc, Y, model_cost, model_output) )






#The variable `expressions` contains the expression estimate for the `sequences`


IOError: File ../pTpA_Glu/models/best_model.ckpt.meta does not exist.